# Bibliotecas

In [1]:
import itertools
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import mac_morpho
from nltk.tag import UnigramTagger
from nltk import pos_tag
from nltk.stem import RSLPStemmer

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('rslp')
nltk.download('stopwords')
nltk.download('mac_morpho')

stemmer = RSLPStemmer()

[nltk_data] Downloading package punkt to C:\Users\Diego
[nltk_data]     Henrique\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Diego
[nltk_data]     Henrique\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package rslp to C:\Users\Diego
[nltk_data]     Henrique\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Diego
[nltk_data]     Henrique\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package mac_morpho to C:\Users\Diego
[nltk_data]     Henrique\AppData\Roaming\nltk_data...
[nltk_data]   Package mac_morpho is already up-to-date!


# Arvore - Estrutura

In [2]:
class Node:
    """
    Representa um nó com um valor e filhos.
    """
    def __init__(self, value):
        """
        Inicializa o nó com um valor e uma lista de filhos vazia.
        """
        self.value = value
        self.children = []

In [3]:
class Tree:
    """
    Representa uma árvore com um nó raiz, permitindo a adição de filhos,
    buscas e operações estruturais.

    Attributes:
        root (Node): O nó raiz da árvore.
    """

    def __init__(self, value):
        """
        Inicializa a árvore com um nó raiz.

        Args:
            value: O valor do nó raiz.
        """
        self.root = Node(value)

    def add_child(self, parent_value, value, stem=False):
        """
        Adiciona um nó filho a um nó pai identificado por seu valor.

        Args:
            parent_value: O valor do nó pai onde o filho será adicionado.
            value: O valor do novo nó filho.
            stem (bool): Indica se o valor deve ser reduzido ao radical (stemming).
        """
        if stem:
            value = stemmer.stem(value)

        parent_node = self.search(parent_value)
        if parent_node:
            parent_node.children.append(Node(value))
        else:
            print(f"Parent node with value {parent_value} not found.")

    def search(self, value, node=None):
        """
        Busca um nó na árvore com base no valor.

        Args:
            value: O valor a ser buscado.
            node (Node): O nó atual da busca. Padrão é o nó raiz.

        Returns:
            Node: O nó correspondente ao valor, ou None se não encontrado.
        """
        if node is None:
            node = self.root

        if node.value == value or node.value == stemmer.stem(value):
            return node

        for child in node.children:
            result = self.search(value, child)
            if result:
                return result
        return None

    def print(self, node=None, level=0):
        """
        Imprime a estrutura da árvore em formato hierárquico.

        Args:
            node (Node): O nó atual para impressão. Padrão é o nó raiz.
            level (int): O nível atual de profundidade na árvore.
        """
        if node is None:
            node = self.root

        print("  " * level + str(node.value))
        for child in node.children:
            self.print(child, level + 1)

    def find_answer(self, entity, intent):
        """
        Encontra a resposta na árvore com base em uma entidade e intenção.

        Args:
            entity (str): A entidade a ser buscada.
            intent (str): A intenção a ser buscada como filha da entidade.

        Returns:
            str: O valor associado ao nó resposta, ou None se não encontrado.
        """
        entity_stem = stemmer.stem(entity)
        intent_stem = stemmer.stem(intent)

        entity_node = self.search(entity_stem)
        if entity_node:
            for child in entity_node.children:
                if child.value == intent_stem:
                    return child.children[0].value
        return None

    def search_in_level(self, value, level, current_level=0, node=None):
        """
        Busca um nó com base no valor em um nível específico da árvore.

        Args:
            value: O valor a ser buscado.
            level (int): O nível da árvore onde buscar.
            current_level (int): O nível atual da busca. Padrão é 0.
            node (Node): O nó atual da busca. Padrão é o nó raiz.

        Returns:
            Node: O nó correspondente ao valor no nível especificado, ou None se não encontrado.
        """
        if node is None:
            node = self.root

        # Se o nível atual for o nível desejado, faz a busca neste nível
        if current_level == level:
            if node.value == value or node.value == stemmer.stem(value):
                return node
            return None

        # Se não está no nível correto, percorre os filhos
        for child in node.children:
            result = self.search_in_level(value, level, current_level + 1, child)
            if result:
                return result

        return None


# Arvore - Dados do Chatbot

In [4]:
tree = Tree("personagens")

# Nós principais
tree.add_child("personagens", "Batman", True)
tree.add_child("personagens", "Flash", True)
tree.add_child("personagens", "Cassandra", True)
tree.add_child("personagens", "Deadpool", True)
tree.add_child("personagens", "Wolverine", True)
tree.add_child("personagens", "quantidade", True)


# Batman
tree.add_child("Batman", "nome", True)
tree.add_child("nome", "O nome verdadeiro do Batman é Bruce Wayne, um bilionário, filantropo e gênio.")

tree.add_child("Batman", "idade", True)
tree.add_child("idade", "Bruce Wayne tem aproximadamente 35 anos de idade nos quadrinhos, embora isso varie dependendo da adaptação.")

tree.add_child("Batman", "historia", True)
tree.add_child("historia", "Bruce Wayne perdeu seus pais, Thomas e Martha Wayne, aos 8 anos de idade em um assalto, evento que o motivou a se tornar o vigilante conhecido como Batman.")

#Flash
tree.add_child("Flash", "poderes", True)
tree.add_child("poderes", "O Flash possui o poder da supervelocidade, permitindo-lhe correr mais rápido que a luz, viajar no tempo e acessar a Força de Aceleração.")

tree.add_child("Flash", "inimigos", True)
tree.add_child("inimigos", "Os principais inimigos do Flash incluem Flash Reverso, Capitão Frio, Gorila Grodd e Mestre dos Espelhos.")

tree.add_child("Flash", "fraqueza", True)
tree.add_child("fraqueza", "A fraqueza do Flash está na sua dependência da Força de Aceleração. Se desconectado dela, ele perde seus poderes.")


# Cassandra
tree.add_child("Cassandra", "habilidades", True)
tree.add_child("habilidades", "Cassandra possui habilidades telepáticas avançadas, incluindo a capacidade de ler mentes, projetar pensamentos e influenciar as ações de outras pessoas.")
tree.add_child("Cassandra", "origem", True)
tree.add_child("origem", "A origem de Cassandra é envolta em mistério. Ela é frequentemente associada à família de mutantes liderada pelo Professor Xavier, mas detalhes sobre seu passado são escassos e pouco explorados.")
tree.add_child("Cassandra", "irmão", True)
tree.add_child("irmão", "O irmão de Cassandra é o Professor Charles Xavier, líder dos X-Men, com quem ela compartilha uma conexão telepática e um histórico conflituoso.")

# Deadpool
tree.add_child("Deadpool", "doença", True)
tree.add_child("doença", "A doença de Deadpool é um câncer terminal que foi tratado com um experimento, resultando em sua imortalidade e regeneração acelerada.")
tree.add_child("Deadpool", "capacidade", True)
tree.add_child("capacidade", "Deadpool possui uma capacidade de regeneração extremamente avançada, que o torna praticamente imortal e capaz de sobreviver a ferimentos fatais.")
tree.add_child("Deadpool", "amigos", True)
tree.add_child("amigos", "Deadpool tem Wolverine como um de seus principais aliados em algumas histórias, além de amizades ocasionais com Cable e Domino.")

# Wolverine
tree.add_child("Wolverine", "armas", True)
tree.add_child("armas", "O Wolverine utiliza suas garras revestidas de adamantium como arma principal, além de sua força sobre-humana e instintos aguçados.")
tree.add_child("Wolverine", "familia", True)
tree.add_child("familia", "O Wolverine teve uma infância conturbada, marcada por perdas e tragédias, incluindo a morte de seus pais biológicos e adotivos. Essas experiências moldaram sua personalidade solitária.")
tree.add_child("Wolverine", "anos", True)
tree.add_child("anos", "O Wolverine possui mais de 100 anos de vida, graças ao seu fator de cura que retarda significativamente seu envelhecimento.")

# Quantidade
tree.add_child("quantidade", "total", True)
tree.add_child("total", "f:consulta")


tree.print()

personagens
  batman
    nom
      O nome verdadeiro do Batman é Bruce Wayne, um bilionário, filantropo e gênio.
    idad
      Bruce Wayne tem aproximadamente 35 anos de idade nos quadrinhos, embora isso varie dependendo da adaptação.
    hist
      Bruce Wayne perdeu seus pais, Thomas e Martha Wayne, aos 8 anos de idade em um assalto, evento que o motivou a se tornar o vigilante conhecido como Batman.
  flash
    pod
      O Flash possui o poder da supervelocidade, permitindo-lhe correr mais rápido que a luz, viajar no tempo e acessar a Força de Aceleração.
    inimig
      Os principais inimigos do Flash incluem Flash Reverso, Capitão Frio, Gorila Grodd e Mestre dos Espelhos.
    fraqu
      A fraqueza do Flash está na sua dependência da Força de Aceleração. Se desconectado dela, ele perde seus poderes.
  cassandr
    habil
      Cassandra possui habilidades telepáticas avançadas, incluindo a capacidade de ler mentes, projetar pensamentos e influenciar as ações de outras pessoas.
  

# Dados para a consulta

In [5]:
import pandas as pd

df = pd.DataFrame({
    'Personagem': ['Batman', 'Flash', 'Cassandra', 'Deadpool', 'Wolverine'],
    'Poder': ['Sem poderes', 'Velocidade', 'Telepatia', 'Regeneração', 'Regeneração/Garras']
})

df


,Personagem,Poder
0,Batman,Sem poderes
1,Flash,Velocidade
2,Cassandra,Telepatia
3,Deadpool,Regeneração
4,Wolverine,Regeneração/Garras


# Chatbot

In [6]:
train_sents = mac_morpho.tagged_sents()
tagger = UnigramTagger(train_sents)

In [7]:
def extract_entities_and_intentions(question):
    """
    Extrai entidades e intenções de uma pergunta.

    A função processa a pergunta para remover palavras irrelevantes (stop words),
    tokeniza a frase, e identifica entidades e intenções baseadas na estrutura de uma árvore.

    Args:
        question (str): A pergunta em formato de texto.

    Returns:
        tuple: Uma tupla contendo duas listas:
            - entities: Lista de entidades identificadas no nível 1 da árvore.
            - intention: Lista de intenções identificadas no nível 2 da árvore.
    """
    stop_words = set(stopwords.words("portuguese"))
    tokens = word_tokenize(question.lower())
    tokens_clean = [word for word in tokens if word not in stop_words and word.isalpha()]

    tagged_tokens = tagger.tag(tokens_clean)

    entities = []
    intention = []

    for token, tag in tagged_tokens:
        if tree.search_in_level(stemmer.stem(token), 1):
            entities.append(stemmer.stem(token))
        elif tree.search_in_level(stemmer.stem(token), 2):
            intention.append(stemmer.stem(token))

    return entities, intention


In [8]:
def find_answer_in_tree(question):
    """
    Encontra uma resposta na árvore com base em uma pergunta.

    A função utiliza a extração de entidades e intenções da pergunta para
    buscar uma resposta correspondente na estrutura da árvore.

    Args:
        question (str): A pergunta em formato de texto.

    Returns:
        str: A resposta encontrada na árvore ou uma mensagem padrão caso não seja possível entender a pergunta.
    """
    entities, intentions = extract_entities_and_intentions(question)
    print(f"Entities: {entities}, Intentions: {intentions}")

    entities_intentions = [[x, y] for x, y in itertools.product(entities, intentions)]
    for entity, intention in entities_intentions:
        answer = tree.find_answer(entity, intention)
        if answer:
            return answer
    return "Desculpe, não consegui entender sua pergunta."


In [9]:
def consult(df):
  """
  Realiza uma consulta no dataframe criado com informações dos Personagens.

  Args:
    df: O dataframe criado.

  Return:
    strf: Retorna a quantidade de personagens existentes no dataframe.
  """
  quant = len(df)

  return f"Existem {quant} personagens"

In [10]:
def chatbot(question):
    """
    Simula um chatbot que responde a uma pergunta com base em uma árvore de conhecimento.

    A função recebe uma pergunta, busca uma resposta na árvore de conhecimento
    utilizando `find_answer_in_tree`, e retorna a resposta apropriada.
    Caso a resposta comece com "f:", uma consulta externa é realizada.

    Args:
        question (str): A pergunta feita pelo usuário.

    Returns:
        None: A resposta é exibida no console.
    """
    print(f"Person: {question}")
    answer = find_answer_in_tree(question)

    if answer.startswith("f:"):
        print(f"Chat: {consult(df)}")
    else:
        print(f"Chat: {answer}")


In [11]:
questions = ["Quem é o irmão da Cassandra?", "Qual é quantidade total de personagens no banco da dados?"]
for question in questions:
  chatbot(question)

Person: Quem é o irmão da Cassandra?
Entities: ['cassandr'], Intentions: ['irm']
Chat: O irmão de Cassandra é o Professor Charles Xavier, líder dos X-Men, com quem ela compartilha uma conexão telepática e um histórico conflituoso.
Person: Qual é quantidade total de personagens no banco da dados?
Entities: ['quant'], Intentions: ['total']
Chat: Existem 5 personagens
